# psychropyo:  Psychrometry using Pyomo
<img src="psychrometricchart.png" width="450" align="right"/>

For Pyomo, see:  http://www.pyomo.org/
   
The psychropyo package is a tool for psychrometry calculations.
It is based on the Pyomo mathematical modeling tool.  
The psychropyo package contains these modules:

* psychrothermo: Pyomo models for thermodynamic properties for air and water   
* psychrostream: Pyomo models for streams of humid air, water, vapor and heat  
* psychroprocess: Pyomo models for simple processes: mix, split, condense, exchange, flowsheetsection, flowsheet    
* psychrochart: module for plotting on a psychrometric chart    
* psychroutils: utilities functions

### Check  kernel version
Should be 3.6 or higher, otherwise, restart with right kernel

In [ ]:
import sys
sys.version_info

In [ ]:
import subprocess

try:
    import pyomo
except ImportError:
    print("Installing Pyomo")
    print(subprocess.run("pip install pyomo", shell=True))
    print(subprocess.run("conda install ipopt_bin -c cachemeorg", shell=True))
    print(subprocess.run("conda install glpk -c cachemeorg", shell=True))

try:
    import holoviews
except ImportError:
    print("Installing HoloViews")
    print(subprocess.run("conda install -c ioam holoviews bokeh", shell=True))

# Psychrometric model with pyomo

In [ ]:
%%time
from psychropyo.psychrochart import *
from psychropyo.psychrostream import *
from psychropyo.psychroprocess import *

columns = ['t','td','tw','rh','pvap','psat','vs','v','h','A','V','W','H','m3']
curves = PsychrometricChart(patm=101325).read()

### flowsheet  with recirculation

In [ ]:
a01  = humid_air("a01", A=1, t=5, rh=0.40)
h1   = heat("h1", H=5)
a02  = humid_air("a02")
heater1 = mix('heater1', a01, h1, a02)

a09  = humid_air("a09")
a03  = humid_air("a03")
recup = mix("recup" , a02, a09  , a03)

h2   = heat("h2")
a04  = humid_air("a04")
heater2 = mix('heater2', a03, h2   , a04)

mist = water("mist" , t=15)
a05  = humid_air("a05")
humid1 = mix("humid1" , a04, mist , a05)

h3   = heat("h3", H=10)
a06  = humid_air("a06")
heater3 = mix('heater3', a05, h3   , a06)

vap  = vapor("vapor", t=37, V=0.003)
a07  = humid_air("a07")
humid2  = mix("humid2" , a06, vap  , a07)

h4   = heat("h4")
a08  = humid_air("a08", t=22, rh=0.40)
losses  = walllosses("losses" , a07, h4 , a08, 1, 5)

a10  = humid_air("a10")
recirc = split("recirc", a08, a09, a10, xb=0.20)

room = flowsheetsection("room", heater3, humid2, losses)
model = flowsheet("model", heater1, recup, heater2, humid1, room, recirc)
#model = flowsheet("model", heater1, recup, heater2, humid1, heater3, humid2, losses, recirc)

model.solve()

Ihtml("<h4>All streams:</h4>")
Idisplay(streamTable(model.streams)[columns])

Ihtml("<h4>Room streams:</h4>")
Idisplay(streamTable(room.streams)[columns])

Ihtml("<h4>Room internal streams:</h4>")
Idisplay(streamTable(room.internalstreams)[columns])

Ihtml("<h4>Psychrometric chart:</h4>")
chart = curves.chart([[a01,a02,a03,a04,a05,a06,a07,a08]])
Idisplay(chart)


In [ ]:
# chart = curves.chart([[a01,a02,a03,a04,a05,a06,a07,a08]], width=877, height=481, xmax=50, ymax=0.020)
# Idisplay(chart)

### flowsheet  with counter current recuperator (simple model)

In [ ]:
a01  = humid_air("a01", A=1, t=5, rh=0.40)
h1   = heat("h1", H=5)
a02  = humid_air("a02")
heater1 = mix('heater1', a01, h1, a02)

a09  = humid_air("a09")
a03  = humid_air("a03")
#recup = mix("recup" , a02, a09  , a03)

h2   = heat("h2")
a04  = humid_air("a04")
heater2 = mix('heater2', a03, h2   , a04)

mist = water("mist" , t=15)
a05  = humid_air("a05")
humid1 = mix("humid1" , a04, mist , a05)

h3   = heat("h3", H=10)
a06  = humid_air("a06")
heater3 = mix('heater3', a05, h3   , a06)

vap  = vapor("vapor", t=37, V=0.003)
a07  = humid_air("a07")
humid2  = mix("humid2" , a06, vap  , a07)

h4   = heat("h4")
a08  = humid_air("a08", t=22, rh=0.40)
losses  = walllosses("losses" , a07, h4 , a08, 1, 5)

a10  = humid_air("a10")
#recirc = split("recirc", a08, a09, a10, xb=0.20)

ccrecup = countercurrent("ccrecup", a08, a10, a02, a03, eff=0.90)  # ok for eff=1, a bit simplistic otherwise

room = flowsheetsection("room", heater3, humid2, losses)
model = flowsheet("model", heater1, ccrecup, heater2, humid1, room)
#model = flowsheet("model", heater1, recup, heater2, humid1, heater3, humid2, losses, recirc)

model.solve()

Ihtml("<h4>All streams:</h4>")
Idisplay(streamTable(model.streams)[columns])

Ihtml("<h4>Room streams:</h4>")
Idisplay(streamTable(room.streams)[columns])

Ihtml("<h4>Room internal streams:</h4>")
Idisplay(streamTable(room.internalstreams)[columns])

Ihtml("<h4>Psychrometric chart:</h4>")
chart = curves.chart([[a01,a02,a03,a04,a05,a06,a07,a08]])
Idisplay(chart)


### Splitting

In [ ]:
ha = humid_air("ha", A=1.0, t=25, rh=0.40)
hb = humid_air("hb")
hc = humid_air("hc")
splitter = split("splitter", ha, hb, hc, xb=0.20)

flowsheet("flowsheet", splitter).solve()

Idisplay(streamTable([ha,hb,hc])[columns])

### condenser

In [ ]:
from pyomo.environ import *
ha = humid_air("ha", A=1, t=25, rh=1.1)
hb = water("hb")
hc = humid_air("hc")
cond = condense("cond", ha, hb, hc)
model = flowsheet("flsh", cond)
model.solve()
df = streamTable([ha,hb,hc])[columns]
Idisplay(df)

### Humid air streams: define, solve, tabulate, plot

In [ ]:
ha = humid_air("ha", patm=101325, A=1, t=25, rh=0.40).solve()
h2 = humid_air("h2", patm=101325, A=2, t=25, rh=0.40).solve()
hb = humid_air("hb", patm=101325, A=2, h=90, rh=1.0).solve()
hc = humid_air("hc", patm=101325, A=1, h=90, rh=0.4).solve()
hd = humid_air("hd", patm=101325, A=1, h=60, rh=0.4).solve()
he = humid_air("he", patm=101325, A=1, h=60, rh=0.8).solve()
hf = humid_air("hf", patm=101325, A=1, h=70, rh=0.3).solve()
hg = humid_air("hg", patm=101325, m3=100, h=70, rh=0.3).solve()

df = streamTable([ha,h2,hb,hc,hd,he,hf,hg,ha])[columns]
Idisplay(df)

chart = curves.chart([[ha,hb],[hb,hd,hc],[he,hf]])
Idisplay(chart)

### Mixing

In [ ]:
ha = humid_air("ha", A=1.0, t=25, rh=0.10)
hb = humid_air("hb", A=0.2, t=45, rh=0.40)
hc = humid_air("hc")
mixer = mix("mixer", ha, hb, hc)

model = flowsheet("flowsheet", mixer)
model.solve()

Idisplay(streamTable([ha,hb,hc])[columns])

chart = curves.chart([[ha,hb,hc]])
Idisplay(chart)

### Humidification, mist

In [ ]:
ha = humid_air("ha", A=1, t=25, rh=0.40)
hb = water("mist", t=25, W=0.001)
hc = humid_air("hc")
mixer = mix("mixer", ha, hb, hc)

flowsheet('humidification mist',mixer).solve()

Idisplay(streamTable([ha,hb,hc])[columns])

### Humidification, vapor

In [ ]:
ha = humid_air("ha", A=1, t=25, rh=0.40)
hb = vapor("vapor", t=25, V=0.002)
hc = humid_air("hc")
mixer = mix("mixer", ha, hb, hc)

flowsheet("flowsheet", mixer).solve()

Idisplay(streamTable([ha,hb,hc])[columns])

curves = PsychrometricChart(patm=101325).read()
chart = curves.chart([[ha,hc]])
Idisplay(chart)

### Heating

In [ ]:
ha = humid_air("ha", A=1, t=25, rh=0.40)
hb = heat("heat", H=10)
hc = humid_air("hc")
mixer = mix("mixer", ha, hb, hc)

flowsheet("flowsheet", mixer).solve()

Idisplay(streamTable([ha,hb,hc])[columns])

curves = PsychrometricChart(patm=101325).read()
chart = curves.chart([[ha,hc]])
Idisplay(chart)

### Heating,  exchange

In [ ]:
ha = humid_air("ha", A=1, t=25, rh=0.40)
hb = heat("heat")
hc = humid_air("hc")
losses = walllosses("losses", ha, hb, hc, 1, 15)

flowsheet("flowsheet", losses).solve()

Idisplay(streamTable([ha,hb,hc])[columns])

curves = PsychrometricChart(patm=101325).read()
chart = curves.chart([[ha,hc]])
Idisplay(chart)

### mix2

In [ ]:
ha = humid_air("ha", A=1, t=25, rh=0.40)
hb = heat("heat", H=10)
hc = humid_air("hc")
hd = humid_air("hd", A=0.2, t=25, rh=0.10)
he = humid_air("he")

mixer1 = mix("mixer1", ha, hb, hc)
mixer2 = mix("mixer2", hc, hd, he)

flowsheet("flowsheet", mixer1,mixer2).solve()

Idisplay(streamTable([ha,hb,hc,hd,he])[columns])

chart = curves.chart([[ha,hc,hd,he]])
Idisplay(chart)